# Extract dynamically downscaled projections

In [1]:
import sys
import os
import glob
import xarray as xr
sys.path.append('/g/data/qz03/wp3/code/git-repo')
from utils import *

In [2]:
import dask
import tempfile
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster

dask.config.set({
    'array.chunk-size': "256 MiB",
    'array.slicing.split_large_chunks': True, #This can make AXIOM very slow
    'distributed.comm.timeouts.connect': '120s',
    'distributed.comm.timeouts.tcp': '120s',
    'distributed.comm.retry.count': 10,
    'distributed.scheduler.allowed-failures': 20,
    "distributed.scheduler.worker-saturation": 1.1, #This should use the new behaviour which helps with memory pile up
})

# client = dask.distributed.Client()
client = Client(n_workers=10, threads_per_worker=1, local_directory = tempfile.mkdtemp(), memory_limit = "63000mb")
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 10
Total threads: 10,Total memory: 586.73 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34029,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:34369,Total threads: 1
Dashboard: /proxy/38717/status,Memory: 58.67 GiB
Nanny: tcp://127.0.0.1:43105,


In [3]:
sys.path.append('/g/data/mn51/users/jb6465/code/hazard-drought-exp/submodules/dataset_finder')
from dataset_finder import *

In [4]:
timescale='mon'
ACS_downscaled_historical = get_datasets("ACS_DS", empat = "historical" , timescale=timescale, var='tas', grid=("AUS-15", "AUS-10i"))#, "AUS-18")) #get_datasets("CORDEX-CMIP6_Regridded", scenario = "historical" , timescale='mon', grid="gr1.5")
ACS_downscaled_ssp370 = get_datasets("ACS_DS", empat = "ssp370" , timescale=timescale, var='tas', grid=("AUS-15", "AUS-10i"))#, "AUS-18")) #get_datasets("CORDEX-CMIP6_Regridded", scenario = "ssp370" , timescale='mon', grid="gr1.5")
ACS_downscaled_ssp126 = get_datasets("ACS_DS", empat = "ssp126" , timescale=timescale, var='tas', grid=("AUS-15", "AUS-10i"))#, "AUS-18")) #get_datasets("CORDEX-CMIP6_Regridded", scenario = "ssp370" , timescale='mon', grid="gr1.5")

In [5]:
ACS_downscaled_historical

,grid,org,gcm,empat,mdl_run,rcm,ver,timescale,,var,date_created,year
0,AUS-15,BOM,ACCESS-CM2,historical,r4i1p1f1,BARPA-R,v1-r1,mon,,"tas, tasmax, tasmin","latest, v20231001",1960 to 2014
1,AUS-15,BOM,ACCESS-ESM1-5,historical,r6i1p1f1,BARPA-R,v1-r1,mon,,"tas, tasmax, tasmin","latest, v20231001",1960 to 2014
2,AUS-15,BOM,CESM2,historical,r11i1p1f1,BARPA-R,v1-r1,mon,,"tas, tasmax, tasmin","latest, v20231001",1960 to 2014
3,AUS-15,BOM,CMCC-ESM2,historical,r1i1p1f1,BARPA-R,v1-r1,mon,,"tas, tasmax, tasmin","latest, v20231001",1960 to 2014
4,AUS-15,BOM,EC-Earth3,historical,r1i1p1f1,BARPA-R,v1-r1,mon,,"tas, tasmax, tasmin","latest, v20231001",1960 to 2014
5,AUS-15,BOM,MPI-ESM1-2-HR,historical,r1i1p1f1,BARPA-R,v1-r1,mon,,"tas, tasmax, tasmin","latest, v20231001",1960 to 2014
6,AUS-15,BOM,NorESM2-MM,historical,r1i1p1f1,BARPA-R,v1-r1,mon,,"tas, tasmax, tasmin","latest, v20231001",1960 to 2014
7,AUS-10i,CSIRO,ACCESS-CM2,historical,r4i1p1f1,CCAM-v2203-SN,v1-r1,mon,,"tas, tasmax, tasmin",v20231206,1951 to 2014
8,AUS-10i,CSIRO,ACCESS-ESM1-5,historical,r6i1p1f1,CCAM-v2203-SN,v1-r1,mon,,"tas, tasmax, tasmin",v20231206,1951 to 2014
9,AUS-10i,CSIRO,CESM2,historical,r11i1p1f1,CCAM-v2203-SN,v1-r1,mon,,"tas, tasmax, tasmin",v20231206,1951 to 2014


In [6]:
%%time
def preprocess_ACS(ds):
    return ds[target_var].sel(lat=slice(domain_dict['indonesia']['lat_min'], domain_dict['indonesia']['lat_max']), lon=slice(domain_dict['indonesia']['lon_min'], domain_dict['indonesia']['lon_max']))


for target_var in ['pr', 'rsds', 'sfcWindmax', 'tas', 'tasmin', 'tasmax']:
    os.makedirs(f'/g/data/qz03/wp3/data/proj/{timescale}/{target_var}', exist_ok=True)
    for ens_idx in range(0,29):    
        if not any(all(sub in filename for sub in ['ssp370',
                                               str(ACS_downscaled_historical[ens_idx]).split('\n')[-4].split()[-1],
                                               str(ACS_downscaled_historical[ens_idx]).split('\n')[-5].split()[-1],
                                               str(ACS_downscaled_historical[ens_idx]).split('\n')[-7].split()[-1]]) for filename in os.listdir(f'/g/data/qz03/wp3/data/proj/{timescale}/{target_var}/')):
            
            print(ACS_downscaled_historical[ens_idx])
            historical_file_list = [file for file in ACS_downscaled_historical[ens_idx].select(var=target_var).get_files() if f'/{target_var}/v' in file]
            ssp126_file_list = [file for file in ACS_downscaled_ssp126[ens_idx].select(var=target_var).get_files() if f'/{target_var}/v' in file]
            ssp370_file_list = [file for file in ACS_downscaled_ssp370[ens_idx].select(var=target_var).get_files() if f'/{target_var}/v' in file]
    
            historical_xr = xr.open_mfdataset(historical_file_list, combine='nested', concat_dim='time', parallel = True, preprocess=preprocess_ACS, engine='netcdf4').compute()
            ssp126_xr = xr.open_mfdataset(ssp126_file_list, combine='nested', concat_dim='time', parallel = True, preprocess=preprocess_ACS, engine='netcdf4').compute()
            ssp370_xr = xr.open_mfdataset(ssp370_file_list, combine='nested', concat_dim='time', parallel = True, preprocess=preprocess_ACS, engine='netcdf4').compute()

            historical_xr.to_netcdf(f"/g/data/qz03/wp3/data/proj/{timescale}/{target_var}/{historical_file_list[0].split('/')[-1][:-17]}.nc",\
                                encoding={f'{target_var}': {'zlib': True, 'complevel': 5, 'dtype':'float32'}})
            ssp126_xr.to_netcdf(f"/g/data/qz03/wp3/data/proj/{timescale}/{target_var}/{ssp126_file_list[0].split('/')[-1][:-17]}.nc",\
                                encoding={f'{target_var}': {'zlib': True, 'complevel': 5, 'dtype':'float32'}})
            ssp370_xr.to_netcdf(f"/g/data/qz03/wp3/data/proj/{timescale}/{target_var}/{ssp370_file_list[0].split('/')[-1][:-17]}.nc",\
                                encoding={f'{target_var}': {'zlib': True, 'complevel': 5, 'dtype':'float32'}})
    
            print()

CPU times: user 5.31 s, sys: 145 ms, total: 5.45 s
Wall time: 5.37 s
